In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

import statsmodels.api as sm

# Dane domów

In [4]:
house = pd.read_csv("data/house_sales.csv", delimiter="\t")

In [5]:
house.head()

,DocumentDate,SalePrice,PropertyID,PropertyType,ym,zhvi_px,zhvi_idx,AdjSalePrice,NbrLivingUnits,SqFtLot,SqFtTotLiving,SqFtFinBasement,Bathrooms,Bedrooms,BldgGrade,YrBuilt,YrRenovated,TrafficNoise,LandVal,ImpsVal,ZipCode,NewConstruction
1,2014-09-16,280000,1000102,Multiplex,2014-09-01,405100,0.930836,300805.0,2,9373,2400,0,3.00,6,7,1991,0,0,70000,229000,98002,False
2,2006-06-16,1000000,1200013,Single Family,2006-06-01,404400,0.929228,1076162.0,1,20156,3764,1452,3.75,4,10,2005,0,0,203000,590000,98166,True
3,2007-01-29,745000,1200019,Single Family,2007-01-01,425600,0.977941,761805.0,1,26036,2060,900,1.75,4,8,1947,0,0,183000,275000,98166,False
4,2008-02-25,425000,2800016,Single Family,2008-02-01,418400,0.961397,442065.0,1,8618,3200,1640,3.75,5,7,1966,0,0,104000,229000,98168,False
5,2013-03-29,240000,2800024,Single Family,2013-03-01,351600,0.807904,297065.0,1,8620,1720,0,1.75,4,7,1948,0,0,104000,205000,98168,False


In [6]:
subset = ['AdjSalePrice', 'SqFtTotLiving', 'SqFtLot', 'Bathrooms', 'Bedrooms', 'BldgGrade']
house[subset].head()

,AdjSalePrice,SqFtTotLiving,SqFtLot,Bathrooms,Bedrooms,BldgGrade
1,300805.0,2400,9373,3.00,6,7
2,1076162.0,3764,20156,3.75,4,10
3,761805.0,2060,26036,1.75,4,8
4,442065.0,3200,8618,3.75,5,7
5,297065.0,1720,8620,1.75,4,7


Zmienne:

* SqFtTotLiving — Całkowita powierzchnia mieszkalna domu
* SqFtLot — Powierzchnia działki
* Bathrooms — Liczba łazienek
* Bedrooms — Liczba sypialni
* BldgGrade — Ocena jakości budynku (1-13)

In [7]:
predictors = ['SqFtTotLiving', 'SqFtLot', 'Bathrooms', 'Bedrooms', 'BldgGrade']
outcome = 'AdjSalePrice'
house_lm = LinearRegression()
house_lm.fit(house[predictors], house[outcome])

LinearRegression()

In [8]:
print(f'Intercept: {house_lm.intercept_:.3f}')
print('Coefficients:')
for name, coef in zip(predictors, house_lm.coef_):
    print(f' {name}: {coef}')

Intercept: -521871.368
Coefficients:
 SqFtTotLiving: 228.83060360240793
 SqFtLot: -0.06046682065307607
 Bathrooms: -19442.840398321066
 Bedrooms: -47769.95518521438
 BldgGrade: 106106.96307898081


In [9]:
fitted = house_lm.predict(house[predictors])
RMSE = np.sqrt(mean_squared_error(house[outcome], fitted))
r2 = r2_score(house[outcome], fitted)
print(f'RMSE: {RMSE:.0f}')
print(f'r2: {r2:.4f}')

RMSE: 261220
r2: 0.5406


In [10]:
import statsmodels.api as sm

X = house[predictors]
X = sm.add_constant(X)
y = house[outcome]

model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           AdjSalePrice   R-squared:                       0.541
Model:                            OLS   Adj. R-squared:                  0.540
Method:                 Least Squares   F-statistic:                     5338.
Date:                Wed, 19 Nov 2025   Prob (F-statistic):               0.00
Time:                        19:47:17   Log-Likelihood:            -3.1517e+05
No. Observations:               22687   AIC:                         6.304e+05
Df Residuals:                   22681   BIC:                         6.304e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const         -5.219e+05   1.57e+04    -33.342      0.000   -5.53e+05   -4.91e+05
SqFtTotLiving   228.8306      3.899     58.694      0.000     221.189     236.472
SqFtLot          -0.0605      0.061     -0.988      0.323      -0.180       0.059
Bathrooms     -1.944e+04   3625.388     -5.363      0.000   -2.65e+04   -1.23e+04
Bedrooms      -4.777e+04   2489.732    -19.187      0.000   -5.27e+04   -4.29e+04
BldgGrade      1.061e+05   2396.445     44.277      0.000    1.01e+05    1.11e+05
==============================================================================
Omnibus:                    29676.557   Durbin-Watson:                   1.247
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         19390738.346
Skew:                           6.889   Prob(JB):                         0.00
Kurtosis:                     145.559   Cond. No.                     2.86e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.86e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Pomysł 1: "Czy model naprawdę działa?" — Porównanie in-sample vs. out-of-sample


Jak to robimy:
* Obliczyć RMSE na pełnych danych (in-sample, jak teraz)
* Zbudować model z train/test split (80/20)
* Obliczyć RMSE na test secie
* Porównać: czy wyniki są podobne czy bardzo się różnią?
* Wniosek: czy model się przefitowuje?

In [12]:
predictors = ['SqFtTotLiving', 'SqFtLot', 'Bathrooms', 'Bedrooms', 'BldgGrade']
outcome = 'AdjSalePrice'

X = house[predictors]
y = house[outcome]

model = LinearRegression()
model.fit(X, y)

rmse_all = np.sqrt(mean_squared_error(y, model.predict(X)))

print(f'RMSE all: {rmse_all:.0f}')

RMSE all: 261220


In [12]:
predictors = ['SqFtTotLiving', 'SqFtLot', 'Bathrooms', 'Bedrooms', 'BldgGrade']
outcome = 'AdjSalePrice'

X = house[predictors]
y = house[outcome]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model_train = LinearRegression()
model_train.fit(X_train, y_train)

rmse_train = np.sqrt(mean_squared_error(y_train, model_train.predict(X_train)))
rmse_test = np.sqrt(mean_squared_error(y_test, model_train.predict(X_test)))

print(f'RMSE training: {rmse_train:.0f}')
print(f'RMSE test: {rmse_test:.0f}')

RMSE training: 258336
RMSE test: 272594


## Pomysł 2: "Jaki model najlepszy?" — k-Fold CV do wyboru modeli

Budujemy kilka modeli i porównują je za pomocą CV:


* Zbuduj 3 modele ze różnymi zestawami predyktorów
* Dla każdego oblicz 5-Fold CV score
* Który model najlepszy? Czy więcej cech zawsze = lepiej?



funcja `cross_val_score()`` — Co to robi?


* Automatycznie wykonuje k-Fold Cross-Validation
* Trenuje i testuje model k razy na różnych podział danych
* Zwraca wyniki (scores) z każdego fold'u
* Eliminujesz ręczne dzielenie danych na train/test

## `scoring='neg_mean_squared_error'` — Metryka oceny

- Oblicza Mean Squared Error dla każdego fold'u
- "neg_" oznacza: zwróć ujemną wartość (konwencja scikit-learn)
- Dlatego później bierzemy pierwiastek i negujemy: np.sqrt(-scores)

In [13]:
from sklearn.model_selection import cross_val_score

# Model 1: mało predyktorów
predictors_1 = ['SqFtTotLiving', 'Bathrooms']
X1 = house[predictors_1]

# Model 2: wszystkie predyktory
predictors_2 = ['SqFtTotLiving', 'SqFtLot', 'Bathrooms', 'Bedrooms', 'BldgGrade']
X2 = house[predictors_2]

# k-Fold CV dla modelu 1
cv_scores_1 = cross_val_score(LinearRegression(), X1, y, 
                              cv=10, scoring='neg_mean_squared_error')
rmse_1 = np.sqrt(-cv_scores_1.mean())

# k-Fold CV dla modelu 2
cv_scores_2 = cross_val_score(LinearRegression(), X2, y, 
                              cv=10, scoring='neg_mean_squared_error')
rmse_2 = np.sqrt(-cv_scores_2.mean())

print(f'Model 1 (mało cech) - RMSE: {rmse_1:.0f}')
print(f'Model 2 (dużo cech) - RMSE: {rmse_2:.0f}')

Model 1 (mało cech) - RMSE: 277433
Model 2 (dużo cech) - RMSE: 261831


## Pomysł 3: "Czy wynik jest niezawodny?" — Stabilność modelu

Sprawdzamy, czy wynik CV jest konsekwentny:



* Uruchom 5-Fold CV

* Porównaj wyniki z każdego fold'u — czy są podobne czy bardzo się różnią?

* Przeanalizuj różnicę między train a test RMSE w każdym fold'u

* Wniosek: czy model jest stabilny? Czy wyniki zależą od podziału danych

funkcja `cross_validate()` — Co to robi?

- Rozszerzona wersja cross_val_score()

- Nie tylko zwraca wyniki, ale też więcej informacji

- Oblicza metryki zarówno na train, jak i na test setach

- Zwraca słownik z pełnymi statystykami

In [14]:
from sklearn.model_selection import cross_validate

# k-Fold CV z pełnymi statystykami
cv_results = cross_validate(LinearRegression(), X2, y, cv=5, 
                           scoring='neg_mean_squared_error', return_train_score=True)

test_scores = np.sqrt(-cv_results['test_score'])
train_scores = np.sqrt(-cv_results['train_score'])

print("RMSE dla każdego fold'u:")
for i, (train, test) in enumerate(zip(train_scores, test_scores)):
    print(f'Fold {i+1}: train={train:.0f}, test={test:.0f}')

print(f'\nŚrednia test RMSE: {test_scores.mean():.0f}')
print(f'Odchylenie std: {test_scores.std():.0f}')

RMSE dla każdego fold'u:
Fold 1: train=265650, test=243016
Fold 2: train=259232, test=269290
Fold 3: train=252714, test=293158
Fold 4: train=256728, test=278565
Fold 5: train=271061, test=217994

Średnia test RMSE: 260405
Odchylenie std: 26768
